In [151]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data


In [152]:
df = pd.read_csv('train_data.csv')
df.head()


,SalePrice,YearBuilt,Size(sqf),Floor,HallwayType,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_manager,N_elevators,SubwayStation,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,141592,2006,814,3,terraced,individual_heating,management_in_trust,111.0,184.0,5min~10min,10min~15min,3.0,0.0,Kyungbuk_uni_hospital,5,6.0,9.0
1,51327,1985,587,8,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
2,48672,1985,587,6,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
3,380530,2006,2056,8,terraced,individual_heating,management_in_trust,249.0,536.0,0~5min,0-5min,5.0,11.0,Sin-nam,5,3.0,7.0
4,78318,1992,644,2,mixed,individual_heating,self_management,142.0,79.0,5min~10min,15min~20min,4.0,8.0,Myung-duk,3,9.0,14.0


In [153]:
print("Shape:", df.shape)
print(df.dtypes)
print()

for column in df.columns:
    # print(df[column].dtype)
    if df[column].dtype not in ("int64", "float64"):
        print(column)
        print(df[column].value_counts())
        print()


Shape: (4124, 17)
SalePrice                      int64
YearBuilt                      int64
Size(sqf)                      int64
Floor                          int64
HallwayType                   object
HeatingType                   object
AptManageType                 object
N_Parkinglot(Ground)         float64
N_Parkinglot(Basement)       float64
TimeToBusStop                 object
TimeToSubway                  object
N_manager                    float64
N_elevators                  float64
SubwayStation                 object
N_FacilitiesInApt              int64
N_FacilitiesNearBy(Total)    float64
N_SchoolNearBy(Total)        float64
dtype: object

HallwayType
terraced    2485
mixed       1194
corridor     445
Name: HallwayType, dtype: int64

HeatingType
individual_heating    3938
central_heating        186
Name: HeatingType, dtype: int64

AptManageType
management_in_trust    3869
self_management         255
Name: AptManageType, dtype: int64

TimeToBusStop
0~5min         3148
5min

In [154]:
def fix_df(df: pd.DataFrame) -> pd.DataFrame:

    # Break time columns to min and max values
    df["TimeToBusStop"] = df["TimeToBusStop"].str.replace("min", "").replace("-", "~")
    df["TimeToBusStop"] = df["TimeToBusStop"].str.replace("-", "~")
    df[["TimeToBusStopMin", "TimeToBusStopMax"]] = df["TimeToBusStop"].str.split('~', 1, expand=True)

    df["TimeToSubway"] = df["TimeToSubway"].str.replace("min", "").replace("-", "~")
    df["TimeToSubway"] = df["TimeToSubway"].str.replace("-", "~")
    df[["TimeToSubwayMin", "TimeToSubwayMax"]] = df["TimeToSubway"].str.split('~', 1, expand=True)

    df.drop(columns=["TimeToBusStop", "TimeToSubway"], inplace=True)

    # One hot encoding for a string columns with a small number of values
    columns_to_onehot = ["HallwayType", "HeatingType", "AptManageType", "SubwayStation"]
    for column in columns_to_onehot:
        y = pd.get_dummies(df[column], prefix=column)
        df[y.columns.to_list()] = y
    df.drop(columns=columns_to_onehot, inplace=True)

    # Convert SalePrice if exists
    if "SalePrice" in df.columns:
        df.loc[df["SalePrice"] <= 100_000, "SalePrice"] = 1
        df.loc[df["SalePrice"] > 100_000, "SalePrice"] = 2
        df.loc[df["SalePrice"] > 350_000, "SalePrice"] = 3

    return df


In [155]:
df = fix_df(df)
df


,SalePrice,YearBuilt,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),N_manager,N_elevators,N_FacilitiesInApt,N_FacilitiesNearBy(Total),...,AptManageType_management_in_trust,AptManageType_self_management,SubwayStation_Bangoge,SubwayStation_Banwoldang,SubwayStation_Chil-sung-market,SubwayStation_Daegu,SubwayStation_Kyungbuk_uni_hospital,SubwayStation_Myung-duk,SubwayStation_Sin-nam,SubwayStation_no_subway_nearby
0,2,2006,814,3,111.0,184.0,3.0,0.0,5,6.0,...,1,0,0,0,0,0,1,0,0,0
1,1,1985,587,8,80.0,76.0,2.0,2.0,3,12.0,...,0,1,0,0,0,1,0,0,0,0
2,1,1985,587,6,80.0,76.0,2.0,2.0,3,12.0,...,0,1,0,0,0,1,0,0,0,0
3,2,2006,2056,8,249.0,536.0,5.0,11.0,5,3.0,...,1,0,0,0,0,0,0,0,1,0
4,1,1992,644,2,142.0,79.0,4.0,8.0,3,9.0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,2,2007,1928,24,0.0,1270.0,14.0,16.0,10,9.0,...,1,0,0,0,0,0,1,0,0,0
4120,2,2015,644,22,102.0,400.0,5.0,10.0,7,7.0,...,1,0,0,0,0,1,0,0,0,0
4121,2,2007,868,20,0.0,1270.0,14.0,16.0,10,9.0,...,1,0,0,0,0,0,1,0,0,0
4122,2,1978,1327,1,87.0,0.0,1.0,4.0,3,7.0,...,0,1,0,0,0,0,1,0,0,0
